In [1]:
%load_ext autoreload
%autoreload 2
from llatb import GameData, Live, DefaultLive, TeamBuilder, html_view, live_basic_data, update_data
from llatb.skill import CenterSkill
pd.options.display.float_format = '{:.3f}'.format
pd.options.display.max_rows = 999
pd.options.display.max_columns = 100

#### Step 0: 使用前请先更新卡牌信息和 Live 信息

In [3]:
update_data()

Creating dictionary for basic live data
Basic live data has been saved in assets/live_data_base.json


#### Step1: 从 tshark json 抓包或者 ieb 数据站导入用户信息（注意抓包文件包含账户信息请不好随意给别人）

In [28]:
# user_profile = GameData('packet.txt', file_type='packet') # Load from tshark json packet
user_profile = GameData('ieb_sample.json', file_type='ieb') # Load from ieb format

Invalid team: 4-th team only has 5 members placed
Invalid team: 5-th team only has 5 members placed
Invalid team: 9-th team only has 8 members placed


#### Step2: 选择想要优化卡组的谱面

In [18]:
group, attr, difficulty = "μ's", 'Cool', 'Master'
group, attr, difficulty = "Aqours", 'Smile', 'Expert'
match_func = lambda x: (x.group==group) & (x.attr==attr) & (x.diff_level==difficulty)
live_basic_data[live_basic_data.apply(match_func, axis=1)]

,attr,diff_level,diff_star,file_dir,group,name,note_number
440,Smile,Expert,10,Live_s0522.json,Aqours,Aqours☆HEROES,515
488,Smile,Expert,10,Live_s0526.json,Aqours,元気全開DAY!DAY!DAY!,592
508,Smile,Expert,10,Live_s0525.json,Aqours,届かない星だとしても,563
520,Smile,Expert,10,Live_s0519.json,Aqours,決めたよHand in Hand,569
524,Smile,Expert,10,Live_s0557.json,Aqours,青空Jumping Heart,471
556,Smile,Expert,10,Live_s0603.json,Aqours,Pops heartで踊るんだもん！,532
564,Smile,Expert,10,Live_s0613.json,Aqours,MIRAI TICKET,536
569,Smile,Expert,9,Live_s0618.json,Aqours,"Guilty Night, Guilty Kiss!",443
629,Smile,Expert,10,Live_s0710.json,Aqours,HAPPY PARTY TRAIN,498
633,Smile,Expert,9,Live_s0714.json,Aqours,太陽を追いかけろ！,476


可以使用默认Live设定, 也可以选择一个存在的谱面

In [6]:
# song_name, difficulty = "Default μ's Smile", 'Master'
# song_name, difficulty = "Default Aqours Smile", 'Expert'
# live = DefaultLive(song_name, difficulty)

song_name, difficulty = '太陽を追いかけろ！', 'Expert'
live = Live(song_name, difficulty)
html_view(live)

#### Step3:  给定应援加成信息，构造 Team Builder 并生成最优卡组

In [10]:
guest_cskill = CenterSkill(name='blah', main_attr='Cool', base_attr='Smile', 
						   main_ratio=12, bonus_range='BiBi', bonus_ratio=6)
opt = {'score_up_bonus':0.1, 'skill_up_bonus':0.1, 'guest_cskill':guest_cskill}
opt = {'score_up_bonus':0, 'skill_up_bonus':0, 'guest_cskill':guest_cskill}
tb = TeamBuilder(live, user_profile, opt=opt)
tb.build_team()

Consider center skill in ['Smile+Smile9%/2nd-year6%', "Smile+Smile9%/μ's3%", 'Smile+Smile9%/1st-year6%', 'Smile+Pure12%/lily white6%', 'Smile+Smile9%/Aqours3%', 'Smile+Cool12%/3rd-year6%', 'Smile+Cool12%/lily white6%', 'Smile+Smile9%/lily white6%']
Best team has score 664886 for Smile+Smile9%/2nd-year6%
Best team has score 653322 for Smile+Smile9%/μ's3%
Best team has score 654090 for Smile+Smile9%/1st-year6%
Best team has score 643207 for Smile+Pure12%/lily white6%
Best team has score 651002 for Smile+Smile9%/Aqours3%
Best team has score 638689 for Smile+Cool12%/3rd-year6%
Best team has score 644101 for Smile+Cool12%/lily white6%
Best team has score 653231 for Smile+Smile9%/lily white6%


如果使用的是存在的谱面，可以进行仿真查看模拟结果，还可以将一次仿真的结果输出成html文件观察

In [8]:
# tb.simulate() # Do not save similation detail to file
tb.simulate(save_to='test.html')

File saved to test.html


将最优卡组导出成为 LL Helper 格式，或 ieb 数据站格式

In [60]:
tb.best_team.to_LLHelper('team.sd')
tb.best_team.to_ieb('team.ieb')

File saved to team.sd
File saved to team.ieb


## 其他功能
可以将用户信息导出为 LL组卡器 支持的格式，使用 LL组卡器 计算最优卡组

In [4]:
user_profile.to_LLTB('cards.666')

file saved to cards.666


查看卡牌信息，排序按照入手时间

In [19]:
html_view(user_profile.raw_card)

,CID,Card View,Skill & Center Skill,,,,,,,skill_gain
,,,,,,,,,,
1,124,,今日は海未の日♪ lv3 40% chance to add 480 score points Triggers every 28 notes. Raise the team's Pure by 6%,80 80,500 500,4,3580,4750,3900,6.8571 pt/note
2,157,,お月見するにゃ！ lv1 30% chance to raise the accuracy judge for 5 seconds Triggers every 29 notes. Raise the team's Pure by 6%,60 60,250 250,3,3590,4130,3730,29.9484% covered
3,81,,幸運の白うさぎ lv1 36% chance to add 180 score points Triggers every 12 perfect notes. Raise the team's Cool by 6%,60 60,250 250,3,3580,3770,4100,5.0837 pt/note
4,167,,ご奉仕してあげる♪ lv1 36% chance to add 570 score points Triggers every 32 combo. Raise the team's Smile by 6%,60 60,250 250,3,4310,3640,3500,6.1560 pt/note
5,174,,読書の秋 lv1 35% chance to raise the accuracy judge for 4 seconds Triggers every 14 seconds. Raise the team's Smile by 6%,60 60,250 250,3,4330,3540,3420,8.4573% covered
6,178,,はい、あーん♪ lv1 28% chance to add 310 score points Triggers every 8 seconds. Raise the team's Smile by 6%,60 60,250 250,3,4460,3540,3390,1.8600 pt/note
7,171,,にこにこサポート lv1 19% chance to restore 3 stamina points Triggers every 8 seconds. Raise the team's Cool by 6%,60 60,250 250,3,3570,3310,4510,0.0122 hp/note
8,55,,笑顔全開！ lv1 36% chance to add 300 score points Triggers every 10 seconds. Raise the team's Smile by 6%,60 60,250 250,3,4330,3730,3230,1.8514 pt/note
9,162,,敏腕マネージャー lv1 13% chance to add 1010 score points Triggers when score reaches a multiple of 13500. Raise the team's Pure by 6%,60 60,250 250,3,3370,4480,3540,13.3099 pt/note


查看卡组信息

In [15]:
html_view(user_profile.team_list[0], show_gem=True)

自定义搜索排序，如将所有红色非特典UR按照(Cool-Pure)的差值降序排列

In [17]:
sel_func = lambda x: x.main_attr=='Smile' and x.promo==False and x.rarity=='UR'
sort_func = lambda x: x.cool-x.pure
df = user_profile.filter(sel_func=sel_func, sort_func=sort_func, ascend=False)
html_view(df, extra_col=['sort_val'])

,CID,Card View,Skill & Center Skill,,,,,,,skill_gain,sort_val
,,,,,,,,,,,
303,782,,期待に応えます！ lv2 28% chance to add 950 score points Triggers every 17 perfect notes. Raise the team's Smile by 12% of its Cool Raise the Smile of lily white members by 6%.,100 100,1000 1000,5,5310,3870,4700,14.6747 pt/note,830
126,280,,亜里沙も一緒♪ lv1 35% chance to restore 4 stamina points Triggers every 29 notes. Raise the team's Smile by 12% of its Cool Raise the Smile of 3rd-year members by 6%.,80 80,500 500,4,4920,3580,4390,0.0480 hp/note,810
89,408,,パフェ一丁！ lv2 37% chance to add 1075 score points Triggers every 25 perfect notes. Raise the team's Smile by 9% Raise the Smile of 1st-year members by 6%.,100 100,1000 1000,5,5330,4080,4400,14.8304 pt/note,320
183,346,,その日が来るまで lv1 38% chance to restore 3 stamina points Triggers every 12 seconds. Raise the team's Smile by 9% Raise the Smile of 2nd-year members by 6%.,80 80,500 500,4,5040,3790,4090,0.0163 hp/note,300
238,955,,お下がり卒業 lv4 42% chance to add 1475 score points Triggers every 19 combo. Raise the team's Smile by 9% Raise the Smile of Aqours members by 3%.,100 100,1000 1000,5,5340,4240,4320,31.8600 pt/note,80
257,66,,小悪魔ガール lv1 36% chance to add 400 score points Triggers every 15 perfect notes. Raise the team's Smile by 9% Raise the Smile of μ's members by 3%.,80 80,500 500,4,4980,4090,3820,9.0171 pt/note,-270
179,666,,気楽にプール♪ lv1 22% chance to raise the accuracy judge for 5 seconds Triggers every 20 notes. Raise the team's Smile by 9% Raise the Smile of lily white members by 6%.,80 80,500 500,4,5070,4160,3780,29.1446% covered,-380
332,476,,海未ちゃんと猛特訓 lv1 52% chance to restore 4 stamina points Triggers every 20 seconds. Raise the team's Smile by 9% Raise the Smile of μ's members by 3%.,80 80,500 500,4,5020,4200,3730,0.0178 hp/note,-470
94,426,,魔法にかけられて lv1 30% chance to add 450 score points Triggers every 15 notes. Raise the team's Smile by 9% Raise the Smile of μ's members by 3%.,80 80,500 500,4,5010,4200,3700,8.8714 pt/note,-500
